In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

# Function to load and preprocess image data
def load_and_preprocess_data(data_dir):
    X = []
    Z = []

    for flower_type in os.listdir(data_dir):
        flower_path = os.path.join(data_dir, flower_type)
        for img in tqdm(os.listdir(flower_path)):
            img_path = os.path.join(flower_path, img)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            img = cv2.resize(img, (500, 500))
            X.append(np.array(img))
            Z.append(str(flower_type))

    le = LabelEncoder()
    Y = le.fit_transform(Z)
    Y = to_categorical(Y, 5)
    X = np.array(X)
    X = X / 255.0  # Normalize pixel values to [0, 1]

    return X, Y

# Load and preprocess data
data_dir = 'C:\\Users\\Soham\\Desktop\\NewFlowers\\TheRealData'  # Replace with the actual path to your dataset
X, Y = load_and_preprocess_data(data_dir)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Model definition
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu', input_shape=(500, 500, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=96, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate reduction callback
red_lr = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.1)

# Train the model
batch_size = 16
epochs = 12

History = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
                    validation_data=(x_test, y_test), callbacks=[red_lr])


100%|██████████| 500/500 [00:09<00:00, 51.69it/s]


MemoryError: Unable to allocate 1.61 GiB for an array with shape (2310, 500, 500, 3) and data type uint8